In [5]:
from thesispy.elastix_wrapper.runner import run
from thesispy.elastix_wrapper.parameters import Parameters, Collection
from thesispy.definitions import *

from pathlib import Path

In [6]:
%%timeit -n 1 -r 2

params = (
    Parameters.from_base(mesh_size=5, metric="AdvancedNormalizedCorrelation", seed=1, use_mask=True)
    .gomea()
    .multi_resolution(1, r_sched=[5])
    .stopping_criteria(3)
    .instance(Collection.LEARN, 1)
)
result = run(params, Path("../output/" + str(params)), suppress_stdout=True, visualize=False, validate=False)

2023-03-18 11:12:32,031 - Runner - INFO - Running elastix in: ../output/1679134352_learn_1_gomea_30870
2023-03-18 11:15:38,318 - Runner - INFO - Run ended. It took 186.2873 seconds
2023-03-18 11:15:38,321 - Runner - INFO - Running elastix in: ../output/1679134538_learn_1_gomea_320392
2023-03-18 11:18:49,310 - Runner - INFO - Run ended. It took 190.9898 seconds


3min 8s ± 2.35 s per loop (mean ± std. dev. of 2 runs, 1 loop each)
